In [1]:
from bio import Animal, AnimalDraw
from ipywidgets import interact


def animal(seed, length, gradation, straightness, distribution):
    a = Animal.from_params(length, gradation, straightness, distribution, seed)
    a.draw(draw, scale=350)
    return output


interact(animal, seed=(1, 999999), length=(7, 15),
         gradation=(1, 100), straightness=(1, 100), distribution=(50, 100))


interactive(children=(IntSlider(value=500000, description='seed', max=999999, min=1), IntSlider(value=11, desc…

<function __main__.animal(seed, length, gradation, straightness, distribution)>